## Tema 4: Indicadores sintéticos

Antes en ejecutar este código, debe leerse con atención las diapositivas correspondientes al Tema 4 disponibles en el Aula Virtual.

### Ejercicio práctico: replicación de creación del indicador sintético de inversión en equipo ISE

Seguimos el documento metodológico "Revisión de la metodología para el cálculo de los Indicadores
Sintéticos de la Economía Española" en http://serviciosede.mineco.gob.es/Indeco/documentos.aspx

In [ ]:
# Importamos funciones y establecemos configuración general
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.x13 import x13_arima_analysis
plt.style.use('seaborn-whitegrid')
plt.rcParams["figure.figsize"] = [10,4]  # Default figure size
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [ ]:
# Importamos la base de datos (disponible en https://github.com/otoperalias/Coyuntura/blob/main/clases/datos/DatosISE.xlsx)
url_data="https://raw.githubusercontent.com/otoperalias/Coyuntura/main/clases/datos/DatosISE.xlsx" # Con esta url se puede importar directamente
cntr = pd.read_excel(url_data,"CNTr") 

In [ ]:
# Preparar la columna periodo con este formato XXXXQX y la podemos con index
cntr['periodo']=cntr.periodo.astype(str) # convertimos formato texto
cntr['periodo']=cntr.periodo.str[:4]+"Q"+cntr.periodo.str[-1:] # Le doy el formato deseado
cntr['periodo']=pd.to_datetime(cntr.periodo) # Convierto a datetime
cntr.set_index('periodo',inplace=True) # Establezco el la columna con index

In [ ]:
# INDICADORES MENSUALES
ind = pd.read_excel(url_data,"Indicadores")
# Preparar la columna periodo con este formato XXXX-XX y la podemos con index
ind['periodo']=ind.periodo.astype(str) # convertimos formato texto
ind['periodo']=ind.periodo.str[:4]+"-"+ind.periodo.str[-2:] # Le doy el formato deseado
ind['periodo']=pd.to_datetime(ind.periodo) # Convierto a datetime
ind.set_index('periodo',inplace=True)

### Paso 1
Seleccionar los indicadores parciales o elementales que resultan representativos de la evolución del PIB, se anualizan y calculan la correlación entre las tasas de variación de cada indicador y el PIB, seleccionando finalmente aquellos con una correlación mayor.

In [ ]:
# ANUALIZAMOS LAS SERIES
cntra=cntr.resample("A").sum()
inda=ind.resample("A").sum()
inda.dispe=inda.dispe/12  # Se divide porque es un índice
inda.dispe.replace(0,np.nan, inplace=True)
inda.vgee=inda.vgee/12   # Se divide porque es un índice
inda.vgee.replace(0,np.nan, inplace=True)
inda=inda["2005":"2018"] # Periodo con datos anuales completos disponibles
cntra=cntra["2005":"2018"]

In [ ]:
# CALCULAMOS TASAS DE VARIACIÓN ANUAL Y CORRELACIONES
    # Tasas de variación
inda_tv=inda/inda.shift(1)-1
cntra_tv=cntra/cntra.shift(1)-1
    # Correlaciones 
corr_dispe=np.corrcoef(cntra_tv.fbcf1b.dropna(),inda_tv.dispe.dropna())[0,1]
corr_mvc1=np.corrcoef(cntra_tv.fbcf1b.dropna(),inda_tv.mvc1.dropna())[0,1]
corr_vgee=np.corrcoef(cntra_tv.fbcf1b.dropna(),inda_tv.vgee.dropna())[0,1]
print("corr_dispe: ", corr_dispe)
print("corr_mvc1: ", corr_mvc1)
print("corr_vgee: ", corr_vgee)

### Paso 2: Extraemos la serie corregida de estacionalidad y calendario
De cada indicador parcial seleccionado en el paso 1, se extrae la serie corregida de variaciones estacionales y de efectos deterministas (calendario y datos atípicos) (“CVEC”) aplicando el programa TRAMO-SEATS.

In [ ]:
    # 1. DISPE
XPATH=os.chdir(r"C:\WinX13\x13as")
modelo = x13_arima_analysis(endog = ind.dispe.dropna(),x12path=XPATH,trading=True)
deses=modelo.seasadj
deses["2010":"2019"].plot(c="r")
ind.dispe["2010":"2019"].plot(c="b")

In [ ]:
# ¿Por qué son tan parecidas?  Porque la serie está tratada!! En la leyenda pone CVEC

In [ ]:
    # 2. MVC
modelo = x13_arima_analysis(endog = ind.mvc1.dropna(),x12path=XPATH,trading=True)
deses=modelo.seasadj
deses["2010":"2019"].plot(c="r")
ind.mvc1["2010":"2019"].plot(c="b")
ind.mvc2["2010":"2019"].plot(c="g") # Esta está ya en CVEC
plt.legend()
plt.show()
mcvd=deses.copy()

In [ ]:
    # 3. VGEE
# No se necesita ajustar porque ya viene ajustada (CVEC)

### Paso 3
Una vez seleccionados los indicadores parciales, sus respectivas series se expresan en forma de índice con base 2000. Posteriormente, el indicador sintético se obtiene como suma ponderada de los indicadores parciales que lo integran (series CVEC), siendo las ponderaciones proporcionales a la correlación simple existente entre el PIB (la macromagnitud que se pretende medir) y el indicador parcial correspondiente. Las ponderaciones han sido normalizadas de modo que su suma sea igual a la unidad.

In [ ]:
# SERIES EN ÍNDICE CON BASE 2005 (en teoría es 2000, pero nuestro primer periodo es 2005)
i_dispe=ind.dispe/ind.dispe["2005-01-01"]
i_mcv=mcvd/mcvd["2005-01-01"] # notar que uso la serie desestacionalizada
i_vgee=ind.vgee/ind.vgee["2005-01-01"]

In [ ]:
# CREACIÓN DEL ÍNDICE SINTÉTICO (IS)
    #Normalizamos las ponderaciones
corr_dispe_n=corr_dispe/(corr_dispe+corr_mvc1+corr_vgee)
corr_mvc1_n=corr_mvc1/(corr_dispe+corr_mvc1+corr_vgee)
corr_vgee_n=corr_vgee/(corr_dispe+corr_mvc1+corr_vgee)
print(corr_dispe_n+corr_mvc1_n+corr_vgee_n)
    # Creamos índice
IS=corr_dispe_n*i_dispe+corr_mvc1_n*i_mcv+corr_vgee_n*i_vgee
    # Plot para comparar el IS con la macromagnitud que queremos capturar
IS.plot(c='r')
cntr.fbcf1a.plot(c='b',secondary_y=True)
plt.legend()
plt.show()

### Paso 4
Se estima el Indicador Sintético Ajustado, que corrige el problema de escala que aparece como consecuencia de la elevada variabilidad del IS respecto al PIB.

In [ ]:
# Calculo de ISAj: (1) Anualizacion de IS y cálculo de tasas de variación anual
ISan=IS.resample("A").mean()
ISan_tv=ISan/ISan.shift(1)-1

In [ ]:
# Regresión lineal entre cntra_tv.fbcf1b["2006":"2018"] y  ISan_tv
y=cntra_tv.fbcf1a["2006":"2018"]
x1=ISan_tv["2006":"2018"]
x1=sm.add_constant(x1)
results = sm.OLS(y,x1).fit()
results.summary()

In [ ]:
# Cálculo de TMIS y TMISAJ
tmis=IS/IS.shift(1)-1
tmisaj=1/12*0.0168+0.7474*tmis

### Paso 5
Considerando enero de 2000=100 y aplicando sucesivamente las TMISAJt, se obtenen los IS Ajustados (ISAJ). Estos indicadores son, en realidad, el resultado de aplicar un cambio de origen y escala a las tasas de crecimiento de los IS originales, paliando así las diferencias de amplitud observadas entre los IS sin ajustar y las variables contables.

In [ ]:
# ISAJ
ISAJ=IS["2005":"2019"].copy()
ISAJ["2005-01-01"]=100
x=100
for p in ISAJ[1:].index:
    ISAJ[p]=x*(1+tmisaj[p])
    x=ISAJ[p]
    # Por fin lo tenemos y lo podemos dibujar junto a la macromagnitud de referencia
ISAJ.plot()
cntr.fbcf1a["2005":"2019"].plot()  # uso "fbcf1a" porque está ajustada
plt.show()

In [ ]:
# Finalmente, las tasas de variación inter anual
ISAJ_tva=ISAJ/ISAJ.shift(12)-1
fbcf1a_tva=cntr.fbcf1a/cntr.fbcf1a.shift(4)-1
ISAJ_tva.plot(c="g")
fbcf1a_tva["2005":"2019"].plot(c="b")  # uso "fbcf1a" porque está ajustada
plt.show()